In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
frec=np.arange(0.1,2.0,0.1)
per = np.array([0.25,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.25,2.5,2.75,3,3.25,3.5,4,5,6,7,8,9,10.0,11,12])

# --- DIMENSIONS ---
rho = 1
l = 1   # length m
d = 1 # draft m
b = 1   # width m
nabla = l*b*d
g=9.806650

w = (2*np.pi)/per
print(len(per))

32


In [3]:
def read_AB(i,j, fname='pot_flow'):
    
    data = np.loadtxt(f'{fname}.1',skiprows=1)

    flags = np.logical_and( data[:,1]==i, data[:,2]==j) 
    AB = {}
    AB['T'] = data[flags,0]
    AB['A'] = data[flags,3]
    AB['B'] = data[flags,4]
    return AB

def read_RAO(eta_i,beta=0.0, fname='pot_flow'):
    data = np.loadtxt(f'{fname}.6',skiprows=1)

    flags = np.logical_and( data[:,1]==beta, data[:,2]==eta_i) 
    rao = {}
    rao['T'] = data[flags,0]
    rao['mod'] = data[flags,3]
    rao['phs'] = data[flags,4]
    rao['re'] = data[flags,5]
    rao['im'] = data[flags,6]

    return rao

# def plot_AB(i,j,fname=None):
#     AB = read_AB(i,j, fname)

#     fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(8.5, 3.5),
#                         layout="constrained")
    
#     axs[0].plot(AB['T'],(AB['A'])/(rho*B*D*175),color="black",marker="o")
#     axs[0].set_xlabel('T [s]')
#     axs[0].set_ylabel(r'$A_{%d%d}/ \rho \nabla \ [-]$'%(i,j))
#     axs[0].set_ylim(bottom=0)
#     axs[0].set_xlim(right=20)
#     # axs[0].invert_yaxis()  # Flip x-axis for A plot
#     axs[0].grid()

#     axs[1].plot(AB['T'],(AB['B'])/(rho*B*D*175*w),color="black",marker="o")
#     axs[1].set_xlabel('T [s]')
#     axs[1].set_ylabel(r'$B_{%d%d}/ \rho \nabla \omega \ [-]$'%(i,j))
#     axs[1].set_xlim(right=20)
#     axs[1].grid()

# def plot_RAO(i,betas=[0],fname=None):

#     fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(8.5, 3.5),
#                     layout="constrained")
        
#     for beta in betas:
#         rao = read_RAO(i,beta, fname)

#         axs[0].plot(rao['T'],rao['mod'], label='Wave %1.1f[deg]'%beta,color="black",marker="o")
#         axs[0].set_xlabel('T')
#         axs[0].set_ylabel(r'$\eta^{a}_{%d}/ \zeta_a$'%(i))
        

        
#         axs[1].plot(rao['T'],rao['phs'], label='Wave %1.1f[deg]'%beta,color="black",marker="o")
#         axs[1].set_xlabel('T')
#         axs[1].set_ylabel(r'$\eta^{\theta}_{%d} [deg]$'%(i))

#     axs[0].legend()
#     axs[1].legend()
   

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
# params = {
#     'pot_flow': {'B': 10000, 'D': 2 / 13.33, 'label': 'Gunnar Study'},
#     # 'main_study_Stian/main_results/Stian_0/pot_flow': {'B': 14, 'D': 14 / 2, 'label': 'Stian Study'},
#     # 'Main Kristian/lengthresults/Arthur_0/pot_flow': {'B': 20, 'D': 20 / 2, 'label': 'Kristian Study'},
#     # 'lengthresults (1)/lengthresults/Arthur_0/pot_flow': {'B': 16, 'D': 16 / 2, 'label': 'Arthur Study'}
# }
# Set font to Times New Roman
# rcParams['font.family'] = 'Times New Roman'
rcParams['font.family'] = 'cmr10'  # This works in most cases

# Alternative approach (for better compatibility)
rcParams['text.usetex'] = True  # Enables LaTeX rendering
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Computer Modern']
# Set default color palette (e.g., seaborn "husl")
palette = sns.color_palette("tab10", 9)

def plot_combined_AB(pairs, file_names, labels, axis_titles, save_files=None):
    """
    Plot A and B coefficients for multiple (i, j) pairs on separate graphs 
    with customizable labels, colors, and axis titles. Optionally save plots to PDF.
    
    Parameters:
        pairs (list of tuples): List of (i, j) pairs to plot.
        file_names (list of str): List of file names to read data from.
        labels (dict): Dictionary containing custom labels and colors for A and B plots:
            - "A_labels": List of labels for A coefficients.
            - "A_colors": List of colors for A lines.
            - "B_labels": List of labels for B coefficients.
            - "B_colors": List of colors for B lines.
        axis_titles (dict): Dictionary containing custom axis titles:
            - "x_label": X-axis label (same for both plots).
            - "y_A_label": Y-axis label for the A plot.
            - "y_B_label": Y-axis label for the B plot.
        save_files (dict): Dictionary containing file names for saving plots:
            - "A_file": File name for the A plot PDF (e.g., "A_plot.pdf").
            - "B_file": File name for the B plot PDF (e.g., "B_plot.pdf").
    """
    # Plot A coefficients
    fig_A, ax_A = plt.subplots(figsize=(7, 5))
    for idx, (i, j) in enumerate(pairs):
        for fname in file_names:
            # Read AB data for each file
            AB = read_AB(i, j, fname)
            
            # # Get specific B, D, and label for the file
            # B = params[fname]['B']
            # D = params[fname]['D']
            
            # Plot A coefficients
            ax_A.plot(
                AB['T'], (AB['A']) / (rho * nabla), 
                marker="o", markersize = 4, linestyle="-", color=labels['A_colors'][idx],
                label=labels['A_labels'][idx]
            )
    ax_A.set_xlabel(axis_titles['x_label'],fontsize=19)
    ax_A.set_ylabel(axis_titles['y_A_label'],fontsize = 19)
    ax_A.tick_params(axis='y', labelsize=15)  # Set y-axis tick label size
    ax_A.tick_params(axis='x', labelsize=15)  # Set x-axis tick label size

    
    # ax_A.set_xticks(fontsize=15)
    # ax_A.set_yticks(fontsize=15)
    ax_A.legend()
    ax_A.grid(True, linestyle='--', alpha=0.6)

    # Save A plot if filename provided
    if save_files and "A_file" in save_files:
        fig_A.savefig(save_files["A_file"], format="pdf", bbox_inches="tight")
    plt.close(fig_A)  # Close the figure to prevent display during script execution

    # Plot B coefficients
    fig_B, ax_B = plt.subplots(figsize=(7, 5))
    for idx, (i, j) in enumerate(pairs):
        for fname in file_names:
            # Read AB data for each file
            AB = read_AB(i, j, fname)
            
            # Get specific B, D, and label for the file
            # B = params[fname]['B']
            # D = params[fname]['D']
            
            # Plot B coefficients
            ax_B.plot(
                AB['T'], (AB['B']) * (l/2*g)**0.5 / (rho * nabla ), 
                marker="s",markersize=4, linestyle="-", color=labels['B_colors'][idx],
                label=labels['B_labels'][idx]
            )
    ax_B.set_xlabel(axis_titles['x_label'],fontsize=19)
    ax_B.set_ylabel(axis_titles['y_B_label'],fontsize=19)
   # Customize B plot
   # Customize B plot
    
    ax_B.tick_params(axis='y', labelsize=15)  # Set y-axis tick label size
    ax_B.tick_params(axis='x', labelsize=15)  # Set x-axis tick label size
    ax_B.legend()


    ax_B.grid(True, linestyle='--', alpha=0.6)

    # Save B plot if filename provided
    if save_files and "B_file" in save_files:
        fig_B.savefig(save_files["B_file"], format="pdf", bbox_inches="tight")
    plt.close(fig_B)  # Close the figure to prevent display during script execution

# Example of how to call the function with custom labels, colors, and titles
plot_combined_AB(
    pairs=[(2, 2), (8, 8), (14, 14), (20, 20), (26, 26), (32, 32), (38, 38), (44, 44), (50, 50)],  # List of (i, j) pairs to plot
    file_names=['pot_flow'],
    labels={
        "A_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "A_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for A lines
        "B_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "B_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for B lines
    },
    axis_titles={
        "x_label": "Period $T$ [s]",
        "y_A_label": r"$A_{22} / \rho \nabla $ [-]",
        "y_B_label": r"$B_{22} \sqrt{l/2g} / \rho \nabla $ [-]"
    },
    save_files={
        "A_file": "wamit_a22.pdf",
        "B_file": "wamit_b22.pdf",
        "A_file": "wamit_a22.pdf",
    }
)


In [5]:
# ---- HEAVE -----

# Example of how to call the function with custom labels, colors, and titles
plot_combined_AB(
    pairs=[(3, 3), (9, 9), (15, 15), (21, 21), (27, 27), (33, 33), (39, 39), (45, 45), (51, 51)],  # List of (i, j) pairs to plot
    file_names=['pot_flow'],
    labels={
        "A_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "A_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for A lines
        "B_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "B_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for B lines
    },
    axis_titles={
        "x_label": "Period $T$ [s]",
        "y_A_label": r"$A_{33} / \rho \nabla $ [-]",
        "y_B_label": r"$B_{33} \sqrt{l/2g} / \rho \nabla $ [-]"
    },
    save_files={
        "A_file": "wamit_a33.pdf",
        "B_file": "wamit_b33.pdf"
    }
)


In [6]:
# ---- PITCH  (ROLL) -----
def plot_combined_pitch_AB(pairs, file_names, labels, axis_titles, save_files=None):
    """
    Plot A and B coefficients for multiple (i, j) pairs on separate graphs 
    with customizable labels, colors, and axis titles. Optionally save plots to PDF.
    
    Parameters:
        pairs (list of tuples): List of (i, j) pairs to plot.
        file_names (list of str): List of file names to read data from.
        labels (dict): Dictionary containing custom labels and colors for A and B plots:
            - "A_labels": List of labels for A coefficients.
            - "A_colors": List of colors for A lines.
            - "B_labels": List of labels for B coefficients.
            - "B_colors": List of colors for B lines.
        axis_titles (dict): Dictionary containing custom axis titles:
            - "x_label": X-axis label (same for both plots).
            - "y_A_label": Y-axis label for the A plot.
            - "y_B_label": Y-axis label for the B plot.
        save_files (dict): Dictionary containing file names for saving plots:
            - "A_file": File name for the A plot PDF (e.g., "A_plot.pdf").
            - "B_file": File name for the B plot PDF (e.g., "B_plot.pdf").
    """
    # Plot A coefficients
    fig_A, ax_A = plt.subplots(figsize=(7, 5))
    for idx, (i, j) in enumerate(pairs):
        for fname in file_names:
            # Read AB data for each file
            AB = read_AB(i, j, fname)
            
            # Get specific B, D, and label for the file
            # B = params[fname]['B']
            # D = params[fname]['D']
            
            # Plot A coefficients
            ax_A.plot(
                AB['T'], (AB['A']) / (rho * nabla* l), 
                marker="o", markersize = 4, linestyle="-", color=labels['A_colors'][idx],
                label=labels['A_labels'][idx]
            )
    ax_A.set_xlabel(axis_titles['x_label'],fontsize=19)
    ax_A.set_ylabel(axis_titles['y_A_label'],fontsize = 19)
    ax_A.tick_params(axis='y', labelsize=15)  # Set y-axis tick label size
    ax_A.tick_params(axis='x', labelsize=15)  # Set x-axis tick label size

    
    # ax_A.set_xticks(fontsize=15)
    # ax_A.set_yticks(fontsize=15)
    ax_A.legend()
    ax_A.grid(True, linestyle='--', alpha=0.6)

    # Save A plot if filename provided
    if save_files and "A_file" in save_files:
        fig_A.savefig(save_files["A_file"], format="pdf", bbox_inches="tight")
    plt.close(fig_A)  # Close the figure to prevent display during script execution

    # Plot B coefficients
    fig_B, ax_B = plt.subplots(figsize=(7, 5))
    for idx, (i, j) in enumerate(pairs):
        for fname in file_names:
            # Read AB data for each file
            AB = read_AB(i, j, fname)
            
            # # Get specific B, D, and label for the file
            # B = params[fname]['B']
            # D = params[fname]['D']
            
            # Plot B coefficients
            ax_B.plot(
                AB['T'], (AB['B']) * (l/2*g)**0.5 / (rho * nabla * l ), 
                marker="s",markersize=4, linestyle="-", color=labels['B_colors'][idx],
                label=labels['B_labels'][idx]
            )
    ax_B.set_xlabel(axis_titles['x_label'],fontsize=19)
    ax_B.set_ylabel(axis_titles['y_B_label'],fontsize=19)
   # Customize B plot
   # Customize B plot
    
    ax_B.tick_params(axis='y', labelsize=15)  # Set y-axis tick label size
    ax_B.tick_params(axis='x', labelsize=15)  # Set x-axis tick label size
    ax_B.legend()


    ax_B.grid(True, linestyle='--', alpha=0.6)

    # Save B plot if filename provided
    if save_files and "B_file" in save_files:
        fig_B.savefig(save_files["B_file"], format="pdf", bbox_inches="tight")
    plt.close(fig_B)  # Close the figure to prevent display during script execution


# Example of how to call the function with custom labels, colors, and titles
plot_combined_pitch_AB(
    pairs=[(4, 4), (10, 10), (16, 16), (22, 22), (28, 28), (34, 34), (40, 40), (46, 46), (52, 52)],  # List of (i, j) pairs to plot
    file_names=['pot_flow'],
    labels={
        "A_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "A_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for A lines
        "B_labels": ["Module 1", "Module 2", "Module 3", "Module 4", "Module 5", "Module 6", "Module 7", "Module 8", "Module 9"],
        "B_colors": [palette[0], palette[1], palette[2],palette[3], palette[4], palette[5], palette[6], palette[7], palette[8]],  # Colors for B lines
    },
    axis_titles={
        "x_label": "Period $T$ [s]",
        "y_A_label": r"$A_{44} / \rho \nabla l$ [-]",
        "y_B_label": r"$B_{44}\sqrt{l/2g} / \rho \nabla l $ [-]"
    },
    save_files={
        "A_file": "wamit_a44.pdf",
        "B_file": "wamit_b44.pdf"
    }
)